# 对自动反向传播代码进行优化
> 1. 定义函数祖类Function时使用循环而非递归的方法

In [1]:
#更改变量类Variable的功能

import numpy as np
class Variable:
    def __init__(self,data):
        self.data = data        
        self.grad = None
        #新拓展
        self.creator = None         #记录变量的创造函数

        #新拓展
    def set_creator(self,func):
        self.creator = func

        #新拓展
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y = f.input,f.output
            x.grad = f.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)    #设置产生变量的creator属性，不可省略，与Variable类反向传播的第17行有强耦合性
        self.input = input
        #新拓展
        self.output = output       #思考可以省略吗?
        return output
    
    def forward(self,input):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = x*2*gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self,gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [2]:
#使用
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
#a = A(x)
a = Square()(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


> 2. 使用工厂函数去简化函数类实例的生成过程

In [4]:
# 工厂函数
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [5]:
#使用
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)


3.297442541400256


> 3. 简化反向传播的首个步骤，定义y关于其自身的导数：即省略 y.grad = no.array(1.0)

In [2]:
#在变量Variable类的反向传播方法开头增加对其grad属性的判断

class Variable:
    def __init__(self,data):
        self.data = data        
        self.grad = None
        #新拓展
        self.creator = None         #记录变量的创造函数

        #新拓展
    def set_creator(self,func):
        self.creator = func

        #新拓展
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y = f.input,f.output
            x.grad = f.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)

'''下不变'''                
class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)    #设置产生变量的creator属性，不可省略，与Variable类反向传播的第17行有强耦合性
        self.input = input
        #新拓展
        self.output = output       #思考可以省略吗?
        return output
    
    def forward(self,input):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = x*2*gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self,gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

> 4. 只支持ndarray类型

```python
# 在Variable类的初始化函数开始先进行类型判断
class Variable:
    def __init__(self,data):
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
```
```python
# 定义一个函数，将标量转换为ndarray类型
'''
np.isscalar(x) 检查输入 x 是否为标量值（如 int, float 等）
如果是标量，则使用 np.array(x) 将其转换为 NumPy 数组
如果不是标量（即已经是数组），则直接返回原值
这个函数在深度学习框架中很重要，因为它确保了所有数据都以 NumPy 数组的形式进行处理
'''
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self )
        self.input = input
        self.output = output
        return output
```

Q：np.array(3.0)为什么不是nd.array类型，使得print(np.isscalar(np.array(3.0)))输出false？

A：np.array(3.0) 生成的是一个零维数组（0-dimensional array），而不是标量（scalar）。在 NumPy 中，标量是单个数值，而零维数组是包含一个元素的数组。
因此，当你使用 np.isscalar(np.array(3.0)) 时，它会返回 False，因为 np.array(3.0) 是一个数组对象，而不是标量。
